In [1]:
! pip3 install xgboost

In [81]:
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import warnings
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore') 

# XGBoost Baseline

### Agregar Intervalo de confianza para los geo_id

In [82]:
train_values = pd.read_csv('train_values.csv', index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')

In [83]:
train_values_baseline = pd.get_dummies(train_values)
train_values_short1 = pd.read_csv('train_values_short1.csv', index_col='building_id')

In [41]:
xgb_cl = XGBClassifier(random_state=2021, max_depth= 10, n_estimators= 100, verbosity=0)

In [9]:
cross_val_score(xgb_cl, train_values_baseline, train_labels, scoring='f1_micro', cv=10).mean()

[16:16:59] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:21:21] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:25:45] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:30:08] WARNING: /home/conda/feedstoc

0.743945727367367

In [10]:
train_values1 = pd.read_csv('train_values_short1.csv', index_col='building_id')

In [11]:
con_ic = train_values1[['min_geo_id', "max_geo_id"]].merge(train_values, left_index=True, right_index=True)
con_ic.drop(columns=["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"], inplace=True)

In [12]:
xgb_cl = XGBClassifier(random_state=2021, max_depth= 10, n_estimators= 100)

In [13]:
cross_val_score(xgb_cl, con_ic, train_labels, scoring='f1_micro', cv=10).mean()

[17:01:02] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:05:19] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:09:36] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:13:53] WARNING: /home/conda/feedstoc

0.7583163551904007

### Pruebo agregar agrupando las superestrtrucutaras por algun material

In [15]:
mud = ['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_mud_mortar_brick']

rc = ['has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered']

stone = ['has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',  'has_superstructure_cement_mortar_stone']

brick = ['has_superstructure_cement_mortar_brick',  'has_superstructure_mud_mortar_brick']

cement = [ 'has_superstructure_cement_mortar_brick', 'has_superstructure_cement_mortar_stone']

grupos = {
    "mud":mud,
    'rc':rc,
    'stone':stone,
    "cement":cement
}

con_grupos = con_ic.copy(deep=True)
for i in grupos:
    con_grupos[i] = con_ic[grupos[i]].max(axis='columns')

In [18]:
xgb_cl = XGBClassifier(random_state=24, max_depth= 10, n_estimators= 100,
                                  gamma=0.07608257721855993,  min_child_weight= 8.395360004452915, verbosity=0)) 

In [15]:
cross_val_score(xgb_cl, con_ic, train_labels, scoring='f1_micro', cv=10).mean()

0.7595251082066359

In [19]:
cross_val_score(xgb_cl, con_grupos, train_labels, scoring='f1_micro', cv=10).mean()

0.7591452178596393

In [ ]:
con_grupos.drop(columns=["rc", "stone", "cement"], inplace=True)

In [25]:
cross_val_score(xgb_cl, con_grupos, train_labels, scoring='f1_micro', cv=10).mean()

0.7590761366206088

Resultado: claramente empeoro todo asi que abandono la idea

### Borrar los geo_ids 2 y 3

In [9]:
xgb_cl =                      XGBClassifier(random_state=2021, max_depth= 10, n_estimators= 100, verbosity=0)  

In [8]:
cross_val_score(xgb_cl, train_values_baseline, train_labels, scoring='f1_micro', cv=10).mean()

0.7439342157662274

In [9]:
train_values_baseline1 = train_values_baseline.drop(columns=["geo_level_2_id","geo_level_3_id"])

In [13]:
cross_val_score(xgb_cl, train_values_baseline1, train_labels, scoring='f1_micro', cv=10).mean()

0.6994716043883464

### Usar el otro encoding de las superestructuras

In [84]:
superestructures = ['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other']

In [43]:
cross_val_score(xgb_cl, train_values_baseline, train_labels, scoring='f1_micro', cv=15).mean()

0.7442220467067296

In [85]:
ambos_encodings = train_values_short1['superestructure'].to_frame().merge(train_values_baseline,
                                               left_index=True, right_index=True)

In [45]:
cross_val_score(xgb_cl, ambos_encodings, train_labels, scoring='f1_micro', cv=15).mean()

0.7447017080677136

In [86]:
solo_short_1 = ambos_encodings.drop(columns=superestructures)

In [88]:
cross_val_score(xgb_cl, solo_short_1, train_labels, scoring='f1_micro', cv=15).mean()

0.7443563474439278

# XGBoost short 1

In [89]:
le = LabelEncoder()
y = le.fit_transform(y=train_labels.values.ravel())

In [90]:
   xgb_cl1 = XGBClassifier( objective = "multi:softmax", use_label_encoder=False, seed=30,
                         max_depth=9,
                         min_child_weight=6,
                         gamma=0.2554138025988315,
                        colsample_bytree = 0.7773754946860542,
                        subsample = 0.9030471581301206,
                         learning_rate =0.1, 
                         n_estimators=273,
                           verbostiy=0
                         )    

### Geo ids

In [91]:
cross_val_score(xgb_cl1, train_values_short1, y, scoring='f1_micro', cv=15).mean()

0.761006339299506

In [92]:
ambos_encodings = train_values_short1.merge(train_values[["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"]],
                                            left_index=True, right_index=True)

In [93]:
cross_val_score(xgb_cl1, ambos_encodings, y, scoring='f1_micro', cv=15).mean()

0.7619080930459515

In [94]:
solo_geo_levels = ambos_encodings.drop(columns=["min_geo_id", "max_geo_id"])

In [95]:
cross_val_score(xgb_cl1, solo_geo_levels, y, scoring='f1_micro', cv=15).mean()

0.7448053018762995

### Grupos superestructures

In [97]:
cross_val_score(xgb_cl1, train_values_short1, y, scoring='f1_micro', cv=15).mean()

0.761006339299506

In [99]:
mud = ['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_mud_mortar_brick']

rc = ['has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered']

stone = ['has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',  'has_superstructure_cement_mortar_stone']

brick = ['has_superstructure_cement_mortar_brick',  'has_superstructure_mud_mortar_brick']

cement = [ 'has_superstructure_cement_mortar_brick', 'has_superstructure_cement_mortar_stone']

grupos = {
    "mud":mud,
    'rc':rc,
    'stone':stone,
    "cement":cement
}

con_grupos = train_values_short1.copy(deep=True)
for i in grupos:
    con_grupos[i] =  train_values_baseline[grupos[i]].max(axis='columns')

In [100]:
cross_val_score(xgb_cl1, con_grupos, y, scoring='f1_micro', cv=15).mean()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/guido/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-100-c7e1ebeb50a1>", line 1, in <module>
    cross_val_score(xgb_cl1, con_grupos, y, scoring='f1_micro', cv=15).mean()
  File "/home/guido/.local/lib/python3.6/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/home/guido/.local/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 450, in cross_val_score
    error_score=error_score)
  File "/home/guido/.local/lib/python3.6/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/home/guido/.local/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 256, in cross_validate
    for train, test in cv.split(X, y, groups))
  File "/home/guido/.local/lib/python3.6/site-packages/joblib/

TypeError: object of type 'NoneType' has no len()